In [59]:
import pandas as pd
from konlpy.tag import Okt
from kiwipiepy import Kiwi
import re
from emoji import core
import emoji

okt = Okt()
kiwi = Kiwi()

In [24]:
df = pd.read_csv(
    "./data/01_raw/youtube_comments_origin.csv",
    index_col=False 
)

df.head()

,video_id,comment_id,comment_text,like_count,published_at
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z


In [25]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 253125 entries, 0 to 253124
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   video_id      253125 non-null  str  
 1   comment_id    253125 non-null  str  
 2   comment_text  253004 non-null  str  
 3   like_count    253125 non-null  int64
 4   published_at  253125 non-null  str  
dtypes: int64(1), str(4)
memory usage: 9.7 MB


# 2. 데이터 전처리

## 1) datetime 열 생성

In [26]:
df["datetime"] = pd.to_datetime(df["published_at"], format="%Y-%m-%dT%H:%M:%SZ")

In [27]:
print(df["datetime"].dtypes)

datetime64[us]


In [28]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 253125 entries, 0 to 253124
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   video_id      253125 non-null  str           
 1   comment_id    253125 non-null  str           
 2   comment_text  253004 non-null  str           
 3   like_count    253125 non-null  int64         
 4   published_at  253125 non-null  str           
 5   datetime      253125 non-null  datetime64[us]
dtypes: datetime64[us](1), int64(1), str(4)
memory usage: 11.6 MB


## 2) 결측치 처리

In [29]:
df.isna().sum()

video_id          0
comment_id        0
comment_text    121
like_count        0
published_at      0
datetime          0
dtype: int64

In [31]:
null_comments = df[df['comment_text'].isnull()]
print(null_comments.head())

          video_id                  comment_id comment_text  like_count  \
7234   rDFUl2mHIW4  Ugz3BCPtRQJkdc9Q-uV4AaABAg          NaN           6   
8629   rDFUl2mHIW4  UgwErp32xCRSWSjX8AB4AaABAg          NaN           6   
9593   rDFUl2mHIW4  UgzfCHYcXvZIORu989x4AaABAg          NaN           3   
15957  rDFUl2mHIW4  UgzAyjBkzbRQlvTH2314AaABAg          NaN           0   
16982  rDFUl2mHIW4  UgzAuoSB6sE6NCgqHdx4AaABAg          NaN           0   

               published_at            datetime  
7234   2023-09-07T14:54:03Z 2023-09-07 14:54:03  
8629   2023-08-23T04:31:10Z 2023-08-23 04:31:10  
9593   2023-08-21T03:24:04Z 2023-08-21 03:24:04  
15957  2023-08-18T10:19:32Z 2023-08-18 10:19:32  
16982  2023-08-18T09:33:10Z 2023-08-18 09:33:10  


In [10]:
df_new = df.dropna().reset_index(drop=True)
df_new.isna().sum()

video_id        0
comment_id      0
comment_text    0
like_count      0
published_at    0
datetime        0
dtype: int64

In [11]:
df_new

,video_id,comment_id,comment_text,like_count,published_at,datetime
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z,2026-01-19 11:10:22
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32
...,...,...,...,...,...,...
252999,TVqEwvbZP30,Ugwm7jVlfsoBlyx2isB4AaABAg,❤❤❤,1,2025-11-24T09:00:25Z,2025-11-24 09:00:25
253000,TVqEwvbZP30,Ugyz3kSw070Uz9sU0Ld4AaABAg,"truly so thankful for riize, the most amazing ...",188,2025-11-24T09:00:25Z,2025-11-24 09:00:25
253001,TVqEwvbZP30,UgzGeXjlTJAOCqIUNtB4AaABAg,来了啊啊啊,1,2025-11-24T09:00:25Z,2025-11-24 09:00:25
253002,TVqEwvbZP30,Ugy81xNDbdlJVTVRadx4AaABAg,earlyyy!!!,1,2025-11-24T09:00:22Z,2025-11-24 09:00:22


## 3) 클리닝

In [84]:
cleaned_comments = []

for comment in df_new["comment_text"]:
    if pd.isna(comment) or not isinstance(comment, str):
        cleaned_comments.append("")
        continue
    
    # 1. 이모지(그림 문자) 제거
    text = core.replace_emoji(comment,"")
    
    # 2. 한글과 공백만 남기기 (초성 'ㅋㅋㅋ', 영어, 숫자, 특수문자 모두 제거)
    text = re.sub(r'[^가-힣\s]', '', text)
    
    # 3. 불필요한 연속 공백 제거 및 양쪽 끝 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    cleaned_comments.append(text)
    

In [85]:
df_new['cleaned_comment'] = cleaned_comments

In [86]:
print(df_new[['comment_text', 'cleaned_comment']])

                                             comment_text  \
0       그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...   
1                                                  눈물이 난다   
2                                             이 노래도 겁나 좋네   
3       패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...   
4                                     언제 들어도 행복하고 좋은 노래..   
...                                                   ...   
252999                                                ❤❤❤   
253000  truly so thankful for riize, the most amazing ...   
253001                                              来了啊啊啊   
253002                                         earlyyy!!!   
253003                                   RIIZE BRIIZE 뜬다！   

                                          cleaned_comment  
0       그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...  
1                                                  눈물이 난다  
2                                             이 노래도 겁나 좋네  
3          패러블 이제 뉴세돌로 리와인드

In [67]:
df_new.head()

,video_id,comment_id,comment_text,like_count,published_at,datetime,cleaned_comment
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29,그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25,눈물이 난다
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56,이 노래도 겁나 좋네
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z,2026-01-19 11:10:22,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보..."
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32,언제 들어도 행복하고 좋은 노래..


In [68]:
df_new.to_csv(
    "./data/01_raw/processed/youtube_comments_clean.csv"
)

In [69]:
df2 = pd.read_csv(
    "./data/01_raw/processed/youtube_comments_clean.csv",
    index_col=0
)
df2.head()

,video_id,comment_id,comment_text,like_count,published_at,datetime,cleaned_comment
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29,그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25,눈물이 난다
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56,이 노래도 겁나 좋네
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z,2026-01-19 11:10:22,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보..."
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32,언제 들어도 행복하고 좋은 노래..


# 3. 데이터 탐색

In [25]:
df2["video_id"].unique()

<StringArray>
['k5O1F_IS0mU', 'rDFUl2mHIW4', 'fff8P0kYexQ', 'T_gJpnjJsnk', 'EYG4ROejmyI',
 '8L001YT4k6k', '6ZUIwj3FgUY', 'B1ShLiq3EVc', '3_b5oKGndn0', 'iUw3LPM7OBU',
 'TVqEwvbZP30']
Length: 11, dtype: str

# 4. 토크나이징

## 1) 전체 데이터

In [ ]:
word_list = []
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

for sent in df2["comment_text"]:
    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)
    result = okt.pos(clean_sent)
    sub_list = []
    for res in result:
        word = res[0]
        pos = res[1]
        if word in stopwords:
            continue
        if pos == "Noun" and len(word) > 1 :
            sub_list.append(word)
    print(sub_list)
    result_str = " ".join(sub_list)
    word_list.append(sub_list)
    print("="*100)

In [72]:
okt.morphs(df2["comment_text"])

AssertionError: phrase input should be string, not <class 'pandas.Series'>

In [ ]:
stopwords = [
    "키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"
    ]

sentence_words = []
sentence_index = []

for _, row in df2.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and pos == "len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [16]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [17]:
sentence_series.head()

k5O1F_IS0mU    [진짜, 애정, 애정, 아이돌, 처음, 콘서트, 아이돌, 처음, 진짜, 마음, 자신...
k5O1F_IS0mU                                                 [눈물]
k5O1F_IS0mU                                                 [노래]
k5O1F_IS0mU                                        [패러블, 다시, 버전]
k5O1F_IS0mU                                             [언제, 노래]
Name: nouns, dtype: object

In [19]:
sentence_series.to_csv("./data/01_raw/subsets/Full_youtube_comments.csv")

## 2) 이세계 아이돌

### (1) 최초공개무대

In [93]:
df_ISEGYE_01= df[df['video_id'] == 'k5O1F_IS0mU']
df_ISEGYE_01

,video_id,comment_id,comment_text,like_count,published_at
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31 15:59:29
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23 04:29:25
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20 01:52:56
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19 11:10:22
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16 02:53:32
...,...,...,...,...,...
1246,k5O1F_IS0mU,Ugyx5xV_uVyS4FwcMgF4AaABAg,미친킹아,0,2023-12-21 17:37:00
1247,k5O1F_IS0mU,Ugz2PXlYohe9FXhs8Md4AaABAg,캬,0,2023-12-21 17:36:59
1248,k5O1F_IS0mU,UgxWD10WvFprN5Qt8pt4AaABAg,캬,0,2023-12-21 17:36:59
1249,k5O1F_IS0mU,UgyuO6NYpvA-8o4CClB4AaABAg,ㄹㄱㄴ,0,2023-12-21 17:36:59


In [ ]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "리외인드"]

sentence_words = []
sentence_index = []

for _, row in df_ISEGYE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [95]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [96]:
sentence_series.head()

k5O1F_IS0mU    [그냥, 진짜, 애정, 애정, 아이돌, 처음, 콘서트, 아이돌, 처음, 진짜, 마음...
k5O1F_IS0mU                                                 [눈물]
k5O1F_IS0mU                                                 [노래]
k5O1F_IS0mU                                [패러블, 이제, 다시, 버전, 보고]
k5O1F_IS0mU                                             [언제, 노래]
Name: nouns, dtype: object

In [97]:
len(sentence_series)

1251

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/ISEGYE_comments_first.csv")

### (2) 최고 조회수

In [99]:
df_ISEGYE_02 = df[df["video_id"] == "rDFUl2mHIW4"]
df_ISEGYE_02

,video_id,comment_id,comment_text,like_count,published_at
1251,rDFUl2mHIW4,Ugx28i-qwkQl3TMV7B14AaABAg,노래가 너무 좋네요. 응원합니다.,0,2026-02-02 07:18:07
1252,rDFUl2mHIW4,UgyYR0Wqvj7MUomOcJx4AaABAg,2026년도에도 보는 사람 개추,13,2026-02-01 06:32:36
1253,rDFUl2mHIW4,UgzHOtXKRsOuzceieYh4AaABAg,2026년에 듣는 사람,6,2026-01-31 00:35:38
1254,rDFUl2mHIW4,Ugzs2HjOb-1kTUFApQp4AaABAg,"They remind me of hololive, they are vtubers a...",3,2026-01-29 05:32:56
1255,rDFUl2mHIW4,Ugyh3QDtNf48vl3Phbl4AaABAg,Are they all vtubers? Like hololive. I’m inter...,5,2026-01-29 05:30:47
...,...,...,...,...,...
20579,rDFUl2mHIW4,UgxFJ1z5lJ95hJDnqe94AaABAg,큰거오나,0,2023-08-18 04:30:15
20580,rDFUl2mHIW4,Ugxhv_RuSpzasuggjpR4AaABAg,너무좋다,0,2023-08-18 04:30:15
20581,rDFUl2mHIW4,UgwAg59HI_vwZcqDBup4AaABAg,킹아,0,2023-08-18 04:30:13
20582,rDFUl2mHIW4,UgywI8hniF6umfe-YWp4AaABAg,오우,0,2023-08-18 04:30:02


In [100]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_ISEGYE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [103]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [104]:
sentence_series.head()

rDFUl2mHIW4        [노래, 응원]
rDFUl2mHIW4    [년도, 사람, 개추]
rDFUl2mHIW4            [사람]
rDFUl2mHIW4            None
rDFUl2mHIW4            None
Name: nouns, dtype: object

In [105]:
len(sentence_series)

19333

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/ISEGYE_comments_most.csv")

### (3) 최신 컴백

In [107]:
df_ISEGYE_03 = df[df['video_id'] == 'fff8P0kYexQ']
df_ISEGYE_03 

,video_id,comment_id,comment_text,like_count,published_at
20584,fff8P0kYexQ,UgwdWrDpJo2BWg28AWt4AaABAg,나 탈덕한 줄 알았는데 아니었네,4,2026-02-02 02:16:39
20585,fff8P0kYexQ,UgwesPoV9K_9mbPPEEF4AaABAg,근데 베이스 지리긴한다...,1,2026-02-01 16:00:52
20586,fff8P0kYexQ,Ugykhnkil5dRTerVKvd4AaABAg,진짜 내 인생곡ㅠ 이세돌이여 영원하라-! 💜💛💙🩷🩵💚,11,2026-01-31 02:38:16
20587,fff8P0kYexQ,UgzPe-6nC6oAQNLh-JV4AaABAg,아위~할때 개이쁘당,8,2026-01-30 17:28:31
20588,fff8P0kYexQ,UgzwwL3vmd_h0zDHG0Z4AaABAg,그냥 노래가 좋다 엄청 잘어울리네,12,2026-01-30 14:39:48
...,...,...,...,...,...
24424,fff8P0kYexQ,UgwEjRH_BRGD1jT5L6d4AaABAg,드디어,0,2025-11-06 09:00:28
24425,fff8P0kYexQ,Ugzro5UtqNodrpym75V4AaABAg,올라왔구나 드디어,0,2025-11-06 09:00:27
24426,fff8P0kYexQ,Ugwq0jvjea2qY6NhSz94AaABAg,떴다,0,2025-11-06 09:00:27
24427,fff8P0kYexQ,Ugx4iqRswJSVHT4TBjp4AaABAg,❤,2,2025-11-06 09:00:24


In [113]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_ISEGYE_03.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [114]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [115]:
sentence_series.head()

fff8P0kYexQ        [탈덕]
fff8P0kYexQ       [베이스]
fff8P0kYexQ    [진짜, 인생]
fff8P0kYexQ        [아위]
fff8P0kYexQ    [그냥, 노래]
Name: nouns, dtype: object

In [116]:
len(sentence_series)

3845

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/ISEGYE_comments_recent.csv")

## 3. 플레이브

### 1) 최초공개무대

In [118]:
df_PLAVE_01 = df[df['video_id'] == 'T_gJpnjJsnk']
df_PLAVE_01

,video_id,comment_id,comment_text,like_count,published_at
24429,T_gJpnjJsnk,Ugwmr8Ar6EQGs5xhwYh4AaABAg,팬 아닌사람들은 영원히 모르겠지만\r\n저는 플레이브가 각자의 서사를 숨기지 않고 ...,0,2026-02-01 20:47:21
24430,T_gJpnjJsnk,Ugz8V7mzu8AAVYQWhgt4AaABAg,Peak,0,2026-01-31 23:59:56
24431,T_gJpnjJsnk,Ugyk-xIfpb4g-KTfgLl4AaABAg,내 꿈도 저렇게 빛났으면 좋겠다,2,2026-01-31 19:32:15
24432,T_gJpnjJsnk,Ugw2PR0vzU5HNyxxLR54AaABAg,오랜만에 돌아왔다...아국가ㅠㅠ어떻게 데뷔곡이 기다릴게? 어떻게 데뷔곡이 자체제작?...,1,2026-01-27 12:58:09
24433,T_gJpnjJsnk,UgyRzeLz69WVLFQDBcZ4AaABAg,기다릴게 너의 그곳에 내가 닿을 수 있게가 데뷔곡 가사라니.. 그때 멤버들의 간절한...,0,2026-01-24 13:11:14
...,...,...,...,...,...
30444,T_gJpnjJsnk,UgxuNJWBpz8ESubkIRV4AaABAg,I think I'm in love ❤️😍💘,4,2023-03-18 07:11:41
30445,T_gJpnjJsnk,UgwR6k_rzeajbbuVvMF4AaABAg,they give off TXT vibes!! but then motion-capt...,5,2023-03-18 07:09:54
30446,T_gJpnjJsnk,UgyZLFkm-zYoikl5o8x4AaABAg,얘들아!!!!! 무대잘봤고 수고해~ 기특해라,10,2023-03-18 07:01:23
30447,T_gJpnjJsnk,UgywxQjut3ANePQRSpd4AaABAg,woah is this mave male version or what?,6,2023-03-18 06:59:07


In [119]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_PLAVE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [120]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [121]:
sentence_series.head()

T_gJpnjJsnk    [사람, 레이브, 각자, 서사, 숨기, 도움, 생각, 버추, 하나, 타격, 영영, ...
T_gJpnjJsnk                                                 None
T_gJpnjJsnk                                                 None
T_gJpnjJsnk            [국가, 데뷔, 데뷔, 자체, 제작, 심지어, 안무, 요지, 플하, 사랑]
T_gJpnjJsnk                            [데뷔, 가사, 그때, 멤버, 마음, 갑자기]
Name: nouns, dtype: object

In [122]:
len(sentence_series)

6020

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/PLAVE_comments_first.csv")

### 2) 최고 조회수

In [124]:
df_PLAVE_02 = df[df['video_id'] == 'EYG4ROejmyI']
df_PLAVE_02

,video_id,comment_id,comment_text,like_count,published_at
30449,EYG4ROejmyI,UgzVM6ti1F291mPYdvZ4AaABAg,안녕하세요 얼굴천재 차은수입니다😊\r\n플레이브와 함께 해서 영광이에요🙏 \r\n펌...,19493,2024-08-20 13:18:06
30450,EYG4ROejmyI,Ugw0DsglBQ_uBtlt3M14AaABAg,Cutenesss overload🤩,0,2026-02-02 07:17:55
30451,EYG4ROejmyI,UgwQtIcAFWdcsTN-PSF4AaABAg,Hello!!! I’m from Indonesia 🇮🇩 I came here aft...,33,2026-01-31 01:48:10
30452,EYG4ROejmyI,UgwKB5WTzcW5_78I_2h4AaABAg,"푸티비 만들던 라이브 생각날 때마다 보는데, 봐도봐도 기분이 짜릿함☺",0,2026-01-29 07:12:43
30453,EYG4ROejmyI,UgwDF1eVkeLyDkn7tKh4AaABAg,솔직히 처음 플레이브 봤을때는 거부감 들었는데 노래 듣다 보니까 자연스레 응원하게 ...,3,2026-01-26 15:19:40
...,...,...,...,...,...
48501,EYG4ROejmyI,UgwloeBa4UgZ3BXCkqt4AaABAg,💙💜💓❤️🖤,1,2024-08-20 08:30:50
48502,EYG4ROejmyI,Ugyn7a623mn6viqfbll4AaABAg,왐마,1,2024-08-20 08:30:45
48503,EYG4ROejmyI,UgzR8USxVPIorXLWRI54AaABAg,와우!,21,2024-08-20 08:30:42
48504,EYG4ROejmyI,Ugy-CYDwPwISu-4L0gB4AaABAg,ahhhhhhh,1,2024-08-20 08:30:42


In [125]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "레이브"]

sentence_words = []
sentence_index = []

for _, row in df_PLAVE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [126]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [127]:
sentence_series.head()

EYG4ROejmyI    [얼굴, 천재, 은수, 영광, 볼륨]
EYG4ROejmyI                    None
EYG4ROejmyI                    None
EYG4ROejmyI           [티비, 라이브, 기분]
EYG4ROejmyI        [처음, 거부, 노래, 응원]
Name: nouns, dtype: object

In [128]:
len(sentence_series)

18057

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/_comments_most.csv")

### 3) 최신 컴백

In [130]:
df_PLAVE_03 = df[df['video_id'] == '8L001YT4k6k']
df_PLAVE_03

,video_id,comment_id,comment_text,like_count,published_at
48506,8L001YT4k6k,Ugwc9ZGQ_RElvVSl9ih4AaABAg,tui cần cái MV này làm background !!!!,0,2026-02-01 15:13:13
48507,8L001YT4k6k,Ugw77q0MPeoPH3P2J-R4AaABAg,진심 멜로디가 사기,2,2026-02-01 01:49:00
48508,8L001YT4k6k,UgyRrLSmfW70x18zWI94AaABAg,헐랭 님들 키티랑 하미니 생일 똑같아요,1,2026-01-31 13:56:11
48509,8L001YT4k6k,UgzBSHev1NttzeOsun14AaABAg,1:56 ❤,0,2026-01-31 05:55:28
48510,8L001YT4k6k,UgzjQ9AUxUOGucpjl2V4AaABAg,This is my daily dose before sleep 😴🥹,5,2026-01-29 13:54:47
...,...,...,...,...,...
66957,8L001YT4k6k,Ugx34V3BiB-CSnGb7Mh4AaABAg,It released!!!💙💜💗❤️🖤,0,2025-11-10 09:00:24
66958,8L001YT4k6k,UgykGUT8V_0BttbSuaV4AaABAg,啊啊啊啊啊啊啊,0,2025-11-10 09:00:23
66959,8L001YT4k6k,UgxZFCVmllWVjaph3XN4AaABAg,플레이브 귀엽다라는 생각이 멈추지 않는 탓일까?,9,2025-11-10 09:00:22
66960,8L001YT4k6k,Ugwylin0tbXNYudygy14AaABAg,PLAVE really said ‘we’re back’ and dropped ano...,125,2025-11-10 09:00:21


In [131]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_PLAVE_03.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [132]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [134]:
sentence_series.head()

8L001YT4k6k                    None
8L001YT4k6k           [진심, 멜로디, 사기]
8L001YT4k6k    [헐랭, 님들, 키티, 하미, 생일]
8L001YT4k6k                    None
8L001YT4k6k                    None
Name: nouns, dtype: object

In [135]:
len(sentence_series)

18456

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/PLAVE_comments_recent.csv")

## 4. 아이브

### 1) 최고조회수

In [137]:
df_IVE_01 = df[df['video_id'] == '6ZUIwj3FgUY']
df_IVE_01

,video_id,comment_id,comment_text,like_count,published_at
66962,6ZUIwj3FgUY,UgxJvLxGUKrWPAETuL94AaABAg,아이브 IAM노래좋아요.❤,0,2026-02-02 07:58:11
66963,6ZUIwj3FgUY,UgwK5_3kMQKcLyO2i6Z4AaABAg,Congrats on your Grammy !!,0,2026-02-02 07:10:11
66964,6ZUIwj3FgUY,UgxgNb5120j1fXHHLK94AaABAg,this og song just won a grammy. cheers ♥️♥️♥️,7,2026-02-02 01:03:46
66965,6ZUIwj3FgUY,UgzyYrD3CM7JKcXgM3F4AaABAg,1:10 분에 도착합니다 오늘 IVE,1,2026-02-02 00:22:14
66966,6ZUIwj3FgUY,UgwwR8BtuuCO6oDNu_x4AaABAg,0:51,0,2026-02-01 21:03:25
...,...,...,...,...,...
174995,6ZUIwj3FgUY,Ugz1kiW89md5qsLAJB94AaABAg,❤😊,0,2023-04-10 09:00:18
174996,6ZUIwj3FgUY,UgzQ6RnSnxhxR6IZ0YR4AaABAg,"다들 음색 미쳣따,,❤\r\n우리 짱이브 요번 활동도 화이팅!\r\n안전하게 잘 끝...",0,2023-04-10 09:00:18
174997,6ZUIwj3FgUY,Ugw1v6yo4q4cddnE_iZ4AaABAg,😊\r\n오오오오옹\r\n아이브🤍🤍🤍🤍🤍\r\n계속 기다렸당\r\n😭😭😭😭😭😭🤍🤍🤍🤍,1,2023-04-10 09:00:18
174998,6ZUIwj3FgUY,UgyanOZRVaE7R0MjA0V4AaABAg,갓기들 진짜 사랑해ㅜㅜ 너무 예뻐ㅜㅜ,1,2023-04-10 09:00:18


In [138]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_IVE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [139]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [140]:
sentence_series.head()

6ZUIwj3FgUY    [아이브, 노래]
6ZUIwj3FgUY         None
6ZUIwj3FgUY         None
6ZUIwj3FgUY     [도착, 오늘]
6ZUIwj3FgUY         None
Name: nouns, dtype: object

In [141]:
len(sentence_series)

108038

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/IVE_comments_most.csv")

### 2) 최신 컴백

In [143]:
df_IVE_02 = df[df['video_id'] == 'B1ShLiq3EVc']
df_IVE_02

,video_id,comment_id,comment_text,like_count,published_at
175000,B1ShLiq3EVc,UgxVCkjnchBj0gkR1CV4AaABAg,잘못하면 유치해질수 있는 컨셉인데...유치하지않고 키치하면서 판타지스럽게 잘 연출한...,0,2026-02-02 02:10:11
175001,B1ShLiq3EVc,UgxBj1itLGZi_vbQVP94AaABAg,почему-то youtube специально тормозит продвиже...,0,2026-02-01 18:16:10
175002,B1ShLiq3EVc,UgzmAnrbpJIuDU-2L2R4AaABAg,"одни хиты от ive, ждём новые скоро…",2,2026-02-01 18:15:20
175003,B1ShLiq3EVc,UgzDAZEfRi_3M5aFZSp4AaABAg,They're all bad music.,0,2026-02-01 16:01:29
175004,B1ShLiq3EVc,Ugyk5OvVKjF9WhK-_FR4AaABAg,35M 🔜 🚨 Daily reminder to str3@m everyday dives!,6,2026-02-01 14:39:47
...,...,...,...,...,...
200384,B1ShLiq3EVc,UgwxVn6x7m_bZAPgui14AaABAg,꺄악,0,2025-08-25 09:00:17
200385,B1ShLiq3EVc,Ugxlcpt0rMnYK5pFaOh4AaABAg,일찍 옴,0,2025-08-25 09:00:17
200386,B1ShLiq3EVc,UgzD9nJre_c9jaSu9bx4AaABAg,1빠!! 썸네일에 원영이 머리 넘 귀엽!!,2,2025-08-25 09:00:17
200387,B1ShLiq3EVc,UgxkVN4GqZQsRnlz0KN4AaABAg,❤,1,2025-08-25 09:00:16


In [144]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_IVE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [145]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [146]:
sentence_series.head()

B1ShLiq3EVc    [잘못, 컨셉, 키치, 판타지, 연출, 뮤비]
B1ShLiq3EVc                         None
B1ShLiq3EVc                         None
B1ShLiq3EVc                         None
B1ShLiq3EVc                         None
Name: nouns, dtype: object

In [147]:
len(sentence_series)

25389

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/IVE_comments_recent.csv")

## 5. 라이즈

### 1) 최초공개무대

In [149]:
df_RIIZE_01 = df[df['video_id'] == '3_b5oKGndn0']
df_RIIZE_01

,video_id,comment_id,comment_text,like_count,published_at
200389,3_b5oKGndn0,Ugyj6wMbDC740iiWtt94AaABAg,2026에도 오는 브리즈?\r\n이 느낌 잃지말아줘 제발,2,2026-01-30 06:06:53
200390,3_b5oKGndn0,UgzoDzdkzpBmPVFHzGx4AaABAg,7 riize i miss u😭😭,0,2026-01-15 00:04:55
200391,3_b5oKGndn0,Ugx47QlArGl5Tw1NyCF4AaABAg,이때 컨셉으로 돌아와…,0,2026-01-13 17:50:12
200392,3_b5oKGndn0,UgzujHkRhnlG9So-yZt4AaABAg,Handsome riize 😊❤😆😏✌️🫰👋🥰,0,2026-01-04 19:02:04
200393,3_b5oKGndn0,UgzP_xs_1AszU4t59iF4AaABAg,2026년에도...❤,0,2026-01-03 23:40:24
...,...,...,...,...,...
201649,3_b5oKGndn0,UgxzpekCRPwbHvNe_ol4AaABAg,Finally ❤️❤️ chukae riize,10,2023-09-07 10:37:29
201650,3_b5oKGndn0,UgwbQXROYsx7hue3T3F4AaABAg,Congratulations for your debut stage 🥳,142,2023-09-07 10:37:21
201651,3_b5oKGndn0,Ugx2LaA59wCVhokZiD54AaABAg,Cuteeeeee,0,2023-09-07 10:36:58
201652,3_b5oKGndn0,Ugx65pSKBagRtAWpVhp4AaABAg,من اومدمممم,0,2023-09-07 10:36:45


In [150]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_RIIZE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [151]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [152]:
sentence_series.head()

3_b5oKGndn0    [브리, 느낌, 제발]
3_b5oKGndn0            None
3_b5oKGndn0            [컨셉]
3_b5oKGndn0            None
3_b5oKGndn0            None
Name: nouns, dtype: object

In [153]:
len(sentence_series)

1265

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/RIIZE_comments_first.csv")

### 2) 최고 조회수

In [155]:
df_RIIZE_02 = df[df['video_id'] == 'iUw3LPM7OBU']
df_RIIZE_02

,video_id,comment_id,comment_text,like_count,published_at
201654,iUw3LPM7OBU,UgzpysFyiOuu9vSDiMd4AaABAg,In another life Riize is 7.....\r\n#RII7E,1,2026-02-01 10:34:39
201655,iUw3LPM7OBU,Ugwzwlsx2nMXe1KoyDN4AaABAg,"the vibes here are timeless, honestlyyyyyy",0,2026-01-31 13:31:54
201656,iUw3LPM7OBU,UgwKLUPhFAkUZ5WzC2p4AaABAg,watching this again feels like the first time,0,2026-01-31 13:31:00
201657,iUw3LPM7OBU,UgzXEsifbaHcEIID72l4AaABAg,"fresh, youthful, and full of charm",0,2026-01-31 13:29:21
201658,iUw3LPM7OBU,UgzIxvdDdWjzMsoNtQR4AaABAg,this MV still hits the same after two years,0,2026-01-31 13:29:06
...,...,...,...,...,...
238150,iUw3LPM7OBU,UgzC1820HI7R_66CN0l4AaABAg,ROOKIE OF THE YEAR,0,2023-09-04 09:00:43
238151,iUw3LPM7OBU,UgyzJMB4ItFCDo2QzC54AaABAg,❤,1,2023-09-04 09:00:40
238152,iUw3LPM7OBU,UgzA14ijlRm7rGzX6bV4AaABAg,P,0,2023-09-04 09:00:38
238153,iUw3LPM7OBU,UgyyUzmtfUi-kZELVVp4AaABAg,"The song, The concept, The visuals, The choreo...",147,2023-09-04 09:00:35


In [159]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_RIIZE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [160]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [161]:
sentence_series.head()

iUw3LPM7OBU    None
iUw3LPM7OBU    None
iUw3LPM7OBU    None
iUw3LPM7OBU    None
iUw3LPM7OBU    None
Name: nouns, dtype: object

In [162]:
len(sentence_series)

36501

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/RIIZE_comments_most.csv")

### 3) 최신 컴백

In [164]:
df_RIIZE_03 = df[df['video_id'] == 'TVqEwvbZP30']
df_RIIZE_03

,video_id,comment_id,comment_text,like_count,published_at
238155,TVqEwvbZP30,UgwQgzuHZXtzXe-gBod4AaABAg,RIIZE ISTHEBEST ！！！！！！,0,2026-02-02 04:26:02
238156,TVqEwvbZP30,UgyH02NPr3Cej19pjy14AaABAg,"In my opinion, this is the best song🎉",0,2026-02-02 04:20:47
238157,TVqEwvbZP30,Ugyi_vZq7KHEEmwJ7t54AaABAg,Has anyone already passed the key to Sohee? Br...,2,2026-02-02 02:01:36
238158,TVqEwvbZP30,UgxOZ8SqQBN94w7Z6d54AaABAg,이거 레알 커리어하이급인데 왜 다들 몰라주는거지 진심 라이즈 노래중에 탑쓰리로 좋은데,3,2026-02-02 01:23:41
238159,TVqEwvbZP30,UgzQS6lO44-awd2oJqF4AaABAg,"Essa música é tão boa, gente!!! Deveria ser ma...",3,2026-02-01 21:52:39
...,...,...,...,...,...
253120,TVqEwvbZP30,Ugwm7jVlfsoBlyx2isB4AaABAg,❤❤❤,1,2025-11-24 09:00:25
253121,TVqEwvbZP30,Ugyz3kSw070Uz9sU0Ld4AaABAg,"truly so thankful for riize, the most amazing ...",188,2025-11-24 09:00:25
253122,TVqEwvbZP30,UgzGeXjlTJAOCqIUNtB4AaABAg,来了啊啊啊,1,2025-11-24 09:00:25
253123,TVqEwvbZP30,Ugy81xNDbdlJVTVRadx4AaABAg,earlyyy!!!,1,2025-11-24 09:00:22


In [165]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

sentence_words = []
sentence_index = []

for _, row in df_RIIZE_03.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["comment_text"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [166]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [167]:
sentence_series.head()

TVqEwvbZP30                          None
TVqEwvbZP30                          None
TVqEwvbZP30                          None
TVqEwvbZP30    [레알, 커리어, 하이, 진심, 라이즈, 노래]
TVqEwvbZP30                          None
Name: nouns, dtype: object

In [ ]:
sentence_series.to_csv("./data/01_raw/subsets/RIIZE_comments_recent.csv")